In [1]:
import pandas as pd

In [2]:
matching_df = pd.read_csv('output/cellprofiler_matching_map.csv')

In [3]:
print("Do compounds that target BRD4 look similar?")
matching_df.query('Metadata_matching_target=="BRD4" and Modality=="compound"')[
    ["Modality", "Cell", "time", "Metadata_matching_target", "above_q_threshold"]
].reset_index(drop=True).sort_values(by=["Modality", "Cell", "time"])

Do compounds that target BRD4 look similar?


,Modality,Cell,time,Metadata_matching_target,above_q_threshold
1,compound,A549,long,BRD4,False
0,compound,A549,short,BRD4,False
3,compound,U2OS,long,BRD4,False
2,compound,U2OS,short,BRD4,False


In [4]:
cosine_sim = pd.read_csv("output/compound_genetic_perturbation_cosine_similarity.csv")

metadata = (
    pd.read_csv(
        "../metadata/external_metadata/JUMP-Target-1_compound_metadata.tsv",
        sep="\t",
        usecols=["broad_sample", "pert_iname"],
    )
    .merge(
        pd.read_csv(
            "input/JUMP-Target-1_compound_metadata_additional_annotations.tsv",
            sep="\t",
            usecols=["broad_sample", "moa_list"],
        )
    )
    .rename(columns={"broad_sample": "Metadata_broad_sample"})
)

In [5]:
cosine_sim_brd4 = cosine_sim.query('Metadata_matching_target=="BRD4"').merge(metadata, on="Metadata_broad_sample")
cosine_sim_bi2536 = cosine_sim.merge(metadata, on="Metadata_broad_sample").query('pert_iname=="BI-2536"')

In [6]:
print('Cosine similarity of BI-2536 to its gene targets')
cosine_sim_bi2536.sort_values(by="cosine_sim", ascending=False).groupby(["Metadata_matching_target", "Genetic_Perturbation", "Cell"]).cosine_sim.median().reset_index()

Cosine similarity of BI-2536 to its gene targets


,Metadata_matching_target,Genetic_Perturbation,Cell,cosine_sim
0,BRD4,CRISPR,A549,0.377306
1,BRD4,CRISPR,U2OS,0.111332
2,BRD4,ORF,A549,-0.233175
3,BRD4,ORF,U2OS,0.600489
4,PLK1,CRISPR,A549,0.755886
5,PLK1,CRISPR,U2OS,0.787449
6,PLK1,ORF,A549,0.106761
7,PLK1,ORF,U2OS,-0.026110


In [7]:
print('Cosine similarity of BRD4 to the compounds that target it')
cosine_sim_brd4.sort_values(by="cosine_sim", ascending=False).groupby(
    ["pert_iname", "Genetic_Perturbation", "Cell"]
).cosine_sim.median().reset_index()

Cosine similarity of BRD4 to the compounds that target it


,pert_iname,Genetic_Perturbation,Cell,cosine_sim
0,BI-2536,CRISPR,A549,0.377306
1,BI-2536,CRISPR,U2OS,0.111332
2,BI-2536,ORF,A549,-0.233175
3,BI-2536,ORF,U2OS,0.600489
4,PFI-1,CRISPR,A549,0.394729
5,PFI-1,CRISPR,U2OS,-0.043047
6,PFI-1,ORF,A549,-0.176815
7,PFI-1,ORF,U2OS,0.306706
8,TG-101348,CRISPR,A549,0.045648
9,TG-101348,CRISPR,U2OS,0.149345
